In [1]:
import joblib
import numpy as np
import pandas as pd
import librosa
import os


In [2]:
model_queen_presence=joblib.load("queen_presence_model.pkl")
model_anomaly = joblib.load("bee_sound_anomaly_model.pkl")
model_queen_acceptance = joblib.load('queen_acceptance_rf_model.joblib')


In [9]:
audio_path = r"D:\AIDS\3rd year\sem2\Bee Hive Condition Projecct\archive\sound_files\split_audio\2022-06-10--00-44-04_1__segment0_part1.wav"
hive_temp = 37
hive_humidity = 36
weather_temp = 17
weather_humidity = 69


# audio_path = input("Enter the full path of the audio clip (.wav): ").strip('"')
# hive_temp = float(input("Enter hive temperature: "))
# hive_humidity = float(input("Enter hive humidity: "))
# weather_temp = float(input("Enter weather temperature: "))
# weather_humidity = float(input("Enter weather humidity: "))

In [10]:
def extract_audio_features(file_path, n_mfcc=13):
    y, sr = librosa.load(file_path, sr=22050)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    mfccs_mean = np.mean(mfccs.T, axis=0)
    return mfccs_mean

mfcc_features = extract_audio_features(audio_path)
mfcc_names = [f"mfcc_{i+1}" for i in range(len(mfcc_features))]
all_features = np.concatenate(([hive_temp, hive_humidity, weather_temp, weather_humidity], mfcc_features))
all_feature_names = ["hive temp", "hive humidity", "weather temp", "weather humidity"] + mfcc_names
input_df = pd.DataFrame([all_features], columns=all_feature_names)

In [11]:
pred_qp = model_queen_presence.predict(input_df)[0]

# === OUTPUT ===
print("\n🧠 Predictions:")
print(f"🐝 Queen Presence: {'Yes' if pred_qp == 1 else 'No'}")


🧠 Predictions:
🐝 Queen Presence: Yes


In [6]:
if pred_qp==1:
    # randome forest model
    predictions = model_queen_acceptance.predict(input_df)

    print("Predicted Queen Acceptance:", predictions[0])

Predicted Queen Acceptance: 1


In [7]:
pred_anomaly = model_anomaly.predict(input_df)[0] 
if weather_temp >= 45 or hive_temp >= 45:
    pred_anomaly="Temp Rising"
    print(f"🚨 Anomaly Detected in Sound: anomaly")

else:
    print(f"🚨 Anomaly Detected in Sound: {'Yes' if pred_anomaly == -1 else 'No'}")


🚨 Anomaly Detected in Sound: No


In [8]:
print(f"🚨 Anomaly Detected in Sound: {'Yes' if pred_anomaly == -1 else 'No'}")

🚨 Anomaly Detected in Sound: No
